In [ ]:
import networkx as nx #For Cnstructing and working on Graphs
from collections import OrderedDict #For Sorting the Dictionaries
import matplotlib.pyplot as plt #For Graph Visualization
import operator #For Itemgetter() Function as we useto get value from a dictionary key
from networkx import common_neighbors
import pandas as pd #For Data Manipulation
import numpy as np #For Data Manipulation

In [ ]:
# G = nx.Graph()
# G.add_edge(1,2)
# G.add_edge(1,3)
# G.add_edge(2,3)
# G.add_edge(2,4)
# G.add_edge(2,5)
# G.add_edge(3,4)
# G.add_edge(3,5)
# G.add_edge(4,5)
# G.add_edge(5,6)
# G.add_edge(6,7)
# G.add_edge(6,8)
# G.add_edge(7,8)
# G.add_edge(7,9)
# G.add_edge(8,9)
# G.add_edge(8,3)
# G.add_edge(9,3)
# G.add_edge(9,4)
# G.add_edge(9,5)
# G.add_edge(9,10)
# G.add_edge(9,11)
# G.add_edge(10,11)
# G.add_edge(10,12)
# G.add_edge(5,8)
# G.add_edge(5,9)
# G.add_edge(5,10)
# G.add_edge(4,10)
# G.add_edge(12,11)
# nx.draw(G, with_labels=True)

In [ ]:
data = open('p2p-Gnutella08.txt').readlines()
G = nx.DiGraph()
for row in data:
    nodeStrip = row.split()
    G.add_edges_from([(nodeStrip[0], nodeStrip[1])])
G = G.to_undirected()
G = nx.convert_node_labels_to_integers(G)
print("Total edges in undirected graph:",len(G.edges()))
print("Total nodes in undirected graph:",len(G.nodes()))

In [ ]:
def truss_decomposition_nx(G):
  i = 2
  G1 = None
  trns_dict = dict()
  while len(G.edges()) > 0: 
    G = nx.k_truss(G, i)

    if i>2:
      edges = set(list(G1.edges())) - set(list(G.edges()))
      edges = list(edges)
      # print(type(edges))
      trns_dict[i-1] = edges
      # print(edges)
    G1 = G.copy()
    # print(G,' i=', i)
    # print('Truss',i,'Edges:',G.edges())
    # nx.draw(G1, with_labels=True)
    # plt.show()
    i += 1
  return trns_dict
#BlogCatalog

trussness_dict = truss_decomposition_nx(G)

In [ ]:
#print(trussness_dict)

In [ ]:
def get_key(val):
    found = False
    for key, value in trussness_dict.items():
        # print(key)
        # print(value) 
        # print(val)
        if val in value:
            found = True 
            return int(key)
    if found == False:
        return 0
print(get_key((2,4)))

In [ ]:
def query_processing_using_k_truss(G,k,q):
    # visited = ()    # visited is a tuple\
    if k == 0 or k == 1:
        return 0
    elif k == 2:
        return G.edges()
    visited = []
    k_truss_index = 0   #Initializing the k-truss index to 0
    n_q=list(G.neighbors(q))    #Getting the neighbors of the query node
    #print(n_q)
    Communities = []  #List to store the nodes of the community
    trussness_dict = truss_decomposition_nx(G) #Getting the trussness dictionary
    for u in n_q:
        #print(get_key((q,u)))
        #print(get_key((u,q)))
        if (get_key((q,u))>=k or get_key((u,q))>=k) and (q,u) not in visited: #If the nodes are k-truss connected
            k_truss_index+=1
            C1=()
            # C1=[]
            Q=[]
            Q.append((q,u))
            #print("Q",Q)
            visited.append((q,u))
            # visited= tuple(set(visited + (q,u)))
            #print("visited",visited)
            while Q != []:
                (x,y)=Q.pop()
                #print("x",x,"y",y)
                C1=tuple(set(C1 + (x,y)))
                # C1.append((x,y))
                print("C1",C1)
                n_x=list(G.neighbors(x))
                #print("n_x",n_x)
                n_y=list(G.neighbors(y))
                #print("n_y",n_y)
                #print(list(set(n_x) & set(n_y)))
                for z in (list(set(n_x) & set(n_y))):
                    if (get_key((x,z))>=k or get_key((z,x))>=k) and (get_key((y,z))>=k or get_key((z,y))>=k) :
                        if (x,z) not in visited and (z,x) not in visited:
                            Q.append((x,z))
                            visited.append((x,z))
                            #print("Q",Q)
                            #print("visited",visited)
                            # visited=tuple(set(visited + (x,z)))
                        if (y,z) not in visited and (z,y) not in visited:
                            Q.append((y,z))
                            visited.append((y,z))
                            #print("Q",Q)
                            #print("visited",visited)
                            # visited=tuple(set(visited + (y,z)))
            # print(C1)
            Communities.append(C1)
    
    return Communities
print("Communities = ",query_processing_using_k_truss(G,3,5))

In [ ]:
def TCP_Index(G):
    trussness_dict = truss_decomposition_nx(G)
    weight={}
    T={}
    S={}
    for x in G.nodes():
        n_x=list(G.neighbors(x))
        # weight={}
        Gx=nx.Graph()
        Gx.add_edges_from(G.edges(G.neighbors(x)))
        for edge in Gx.edges():
            y=edge[0]
            z=edge[1]
            # temp=[get_key((x,y)),get_key((y,x)),get_key((x,z)),get_key((z,x)),get_key((y,z)),get_key((z,y))]
            temp=[get_key((x,y)),get_key((x,z)),get_key((y,z))]
            weight[(y,z)]=min(temp)

        T[x]=n_x
        kmax=max(weight.values())
        for k in range(kmax, 2):
            if weight[(y,z)]==k:
                S[x]=(y,z)
            for (y,z) in S.values():
                if y not in T[x] and z not in T[x]:
                    T[(y,z)]=weight[(y,z)]
    return G
print(TCP_Index(G))

In [ ]:
def Query_Processing_TCP(G,k,q):
    T=TCP_Index(G)
    visited = []
    l=0
    Communities = []
    n_q=list(G.neighbors(q))
    for u in n_q:
        if (get_key((q,u))>=k or get_key((u,q))>=k) and (q,u) not in visited:
            l+=1
            V={}
            C1=()
            Q=[]
            Q.append((q,u))
            # print("Q",Q)
            while Q != []:   
                (x,y)=Q.pop()
                # print("x",x,"y",y)
                V[tuple([x,y])]=T[x]
                # print("V",V)
                if ((x,y)) not in visited:
                    for i in V.values():
                        for z in i:
                            visited.append((x,z))
                            C1=tuple(set(C1 + (x,z)))
                            if (z,x) not in visited:
                                Q.append((z,x))
            # print(C1)
            Communities.append(C1)
    return Communities
print("Communities = ",Query_Processing_TCP(G,3,5))

In [ ]:
# def Query_Processing_TCP(G,k,q):
#     T=TCP_Index(G)
#     print(T)
#     visited = []
#     l=0
#     Communities = []
#     V={}
#     n_q=list(G.neighbors(q))
#     for u in n_q:
#         if (get_key((q,u))>=k or get_key((u,q))>=k) and (q,u) not in visited:
#             l+=1
#             C1=()
#             Q=[]
#             Q.append((q,u))
#             visited.append((q,u))
#             print("Q",Q)
#             print("visited",visited)
#             while Q != []:   
#                 (x,y)=Q.pop()
#                 print("x",x,"y",y)
#                 if ((x,y)) not in visited:
#                     print(T)
#                     for key,value in T.items():
#                         if [x,y] in value and int(key)>=k:                         
#                             V[(x,y)]=x
#                             print("V",V)
#                     for z in V.values():
#                         visited.append((x,z))
#                         C1=tuple(set(C1 + (x,z)))
#                         if (z,x) not in visited:
#                             Q.append((z,x))
#             Communities.append(C1)
#     return Communities
# print("Communities = ",Query_Processing_TCP(G,3,10))

In [ ]:
# def Query_Processing_TCP(G,q):
#     T=TCP_Index(G)
#     n_q=list(G.neighbors(q))
#     visited=[]
#     Q=[]
#     Q.append(q)
#     visited.append(q)
#     while Q != []:
#         x=Q.pop()
#         n_x=list(G.neighbors(x))
#         for y in n_x:
#             if y not in visited:
#                 Q.append(y)
#                 visited.append(y)
#     Communities=[]
#     for x in visited:
#         if x in T:
#             Communities.append(T[x])
#     return Communities
# print(Query_Processing_TCP(G,5))

In [ ]:
# def query_processing_using_k_truss(G,k,q):
#     visited = []    #List to store the visited nodes
#     k_truss_index = 0   #Initializing the k-truss index to 0
#     n_q=list(G.neighbors(q))    #Getting the neighbors of the query node
#     Communities = []  #List to store the nodes of the community
#     trussness_dict = truss_decomposition_nx(G) #Getting the trussness dictionary
#     for u in n_q:
#         if get_key((q,u))>=k and (q,u) not in visited: #If the nodes are k-truss connected
#             k_truss_index+=1
#             C1=[]
#             Q=[]
#             Q.push((q,u))
#             visited=visited | {q,u}
#             while Q != []:
#                 (x,y)=Q.pop()
#                 C1=C1 | {x,y}
#                 for z in G.neighbors(x) & G.neighbors(y):
#                     if get_key((x,z))>=k and get_key((y,z)) >=k :
#                         if (x,z) not in visited:
#                             Q.push((x,z))
#                             visited=visited | {(x,z)}
#                         if (y,z) not in visited:
#                             Q.push((y,z))
#                             visited=visited | {(y,z)}
#             Communities.append(C1)
    
#     return Communities

In [ ]:
# def query_processing_using_k_truss(G,k,q):
#     visited = []    #List to store the visited nodes
#     k_truss_index = 0   #Initializing the k-truss index to 0
#     n_q=list(G.neighbors(q))    #Getting the neighbors of the query node
#     Communities = []  #List to store the nodes of the community
#     trussness_dict = truss_decomposition_nx(G) #Getting the trussness dictionary
#     for u in n_q:
#         if get_key((q,u))>=k and (q,u) not in visited: #If the nodes are k-truss connected
#             k_truss_index+=1
#             C1=[]
#             Q=[]
#             Q.push((q,u))
#             visited=visited | {q,u}
#             while Q != []:
#                 (x,y)=Q.pop()
#                 C1=C1 | {x,y}
#                 for z in G.neighbors(x) & G.neighbors(y):
#                     if get_key((x,z))>=k and get_key((y,z) >=k :
#                         if (x,z) not in visited:
#                             Q.push((x,z))
#                             visited=visited | {(x,z)}
#                         if (y,z) not in visited:
#                             Q.push((y,z))
#                             visited=visited | {(y,z)}
#             Communities.append(C1)
    
#     return Communities

In [ ]:
# def query_processing_using_k_truss(G,k,q):
#     visited = []    #List to store the visited nodes
#     k_truss_index = 0   #Initializing the k-truss index to 0
#     n_q=list(G.neighbors(q))    #Getting the neighbors of the query node
#     Communities = []  #List to store the nodes of the community
#     for u in n_q:
#         if k_truss_decomposition(G,k,u,q)>=k and (q,u) not in visited: #If the nodes are k-truss connected
#             k_truss_index+=1
#             C1=[]
#             Q=[]
#             Q.push((q,u))
#             visited=visited | {q,u}
#             while Q != []:
#                 (x,y)=Q.pop()
#                 C1=C1 | {x,y}
#                 for z in G.neighbors(x) & G.neighbors(y):
#                     if k_truss_decomposition(G,k,z,x)>=k and k_truss_decomposition(G,k,z,y) >=k :
#                         if (x,z) not in visited:
#                             Q.push((x,z))
#                             visited=visited | {(x,z)}
#                         if (y,z) not in visited:
#                             Q.push((y,z))
#                             visited=visited | {(y,z)}
#             Communities.append(C1)
    
#     return Communities